In [1]:
import librosa
import glob
import numpy as np
import soundfile as sf
import os
import pandas as pd
import random
from copy import copy


In [4]:
df= pd.read_csv('audio_data/meta/esc2.csv')
df['filename'] = df['filename'].apply(lambda x: str(x)+'.wav')
df.to_csv('audio_data/meta/esc2.csv', index=False)

In [18]:

def extract_classes(path):
    df = pd.read_csv(os.path.join(path, 'meta/esc50.csv'))
    list_fire = df[df.target==12]['filename'].tolist()
    list_no_fire = df[df.target!=12]['filename'].tolist()
    return list_fire, list_no_fire

In [19]:
list_fire, list_no_fire = extract_classes('data/esc50')
random.shuffle(list_fire)
random.shuffle(list_no_fire)
#fire split
list_fire_train , list_fire_val_test = list_fire[:len(list_fire)//2], list_fire[len(list_fire)//2:]
# non fire split :
list_non_fire_train , list_non_fire_val_test = list_no_fire[:len(list_no_fire)//2], list_no_fire[len(list_no_fire)//2:]

In [20]:
len(list_non_fire_train)

980

il faut avoir 960 donnée augmenter dans l'entrainement

In [31]:
#changer seuelemnt  ( i_start , label du target )

def add_audio(path ,list_inp):
    i_start=1980

    i = i_start
    for name_audio in list_inp :
        audio, sr = librosa.load(os.path.join(path,'audio', name_audio), sr=None)

        out_path = os.path.join("audio_data/audio", str(i)+'.wav')
        sf.write(out_path, audio, sr)
        i+=1
    
    df = pd.read_csv("audio_data/meta/esc2.csv")
    df1 = pd.DataFrame({'filename': [j for j in range(i_start,len(list_inp)+i_start)],
                        'target' : [0 for j in range(len(list_inp))] })
    df = pd.concat([df, df1])
    df.to_csv("audio_data/meta/esc2.csv", index=False)



In [32]:
# add 20 fire audio for training
add_audio('data/esc50', list_non_fire_val_test)

In [16]:
def delete_audio():

    ###### delete fire audio #####
    # Path to the folder containing the .wav files
    folder_path = "audio_data/audio"

    # Use glob to find all .wav files in the folder and delete them
    wav_files = glob.glob(os.path.join(folder_path, "*.wav"))

    for file_path in wav_files:
        os.remove(file_path)

    #### delete non fire audio ####  
    # Path to the folder containing the .wav files
    folder_path = "audio_data/non_fire_audio"

    # Use glob to find all .wav files in the folder and delete them
    wav_files = glob.glob(os.path.join(folder_path, "*.wav"))

    for file_path in wav_files:
        os.remove(file_path)

delete_audio()

In [33]:

def add_background(valll, path, main_path, bg_path, bg_volume=0.1):
    
    # Load audio files
    main_audio, sr_main = librosa.load(os.path.join(path,'audio', main_path), sr=None)
    bg_audio, sr_bg = librosa.load(os.path.join(path,'audio', bg_path), sr=None)

    # Ensure sample rates match
    if sr_main != sr_bg:
        raise ValueError("Sample rates of main and background audio must match")

    # Ensure both audio signals are the same length
    if len(main_audio) > len(bg_audio):
        # Repeat or truncate the background audio to match the length of the main audio
        bg_audio = np.tile(bg_audio, int(np.ceil(len(main_audio) / len(bg_audio))))[:len(main_audio)]
    elif len(bg_audio) > len(main_audio):
        # Truncate background audio if it's longer
        bg_audio = bg_audio[:len(main_audio)]
    
    # Scale the background audio
    bg_audio = bg_audio * bg_volume
    
    # Combine the main audio and background audio
    mixed_audio = main_audio + bg_audio
    
    # Normalize the mixed audio to prevent clipping
    max_amplitude = np.max(np.abs(mixed_audio))
    if max_amplitude > 1.0:
        mixed_audio = mixed_audio / max_amplitude

    out_path = os.path.join("audio_data/audio", str(valll+2960)+'.wav')

    # Save the mixed audio
    sf.write(out_path, mixed_audio, sr_main)


    df = pd.read_csv("audio_data/meta/esc2.csv")
    df1 = pd.DataFrame({'filename': [valll+2960],
                        'target' : [1] })
    df = pd.concat([df, df1])
    df.to_csv("audio_data/meta/esc2.csv", index=False)



In [34]:


# fire audio augmentation

ls_main = copy(list_fire_val_test)
ls_bg = copy(list_non_fire_val_test)
track= []

for valll in range(960):
    while True: 

        main_path = random.choice(ls_main)
        bg_path = random.choice(ls_bg)
        
        if (main_path, bg_path) not in track : 
            track.append((main_path, bg_path))
            break
        
    add_background(valll, 'data/esc50', main_path, bg_path)




In [ ]:
# indice train : range(1960)


In [35]:
df = pd.read_csv("audio_data/meta/esc2.csv")
df

,filename,target
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
3915,3915,1
3916,3916,1
3917,3917,1
3918,3918,1
